In [97]:
import pandas as pd
import numpy as np

In [98]:
has_parent = pd.read_csv('2022bkk-buget-has_parent.csv')
item = pd.read_csv('2022bkk-buget-item.csv', index_col='id')

In [99]:
df = item.join(has_parent.set_index('id'), on='id')

In [100]:
df.amount = df.amount.astype(np.float32)

In [101]:

psum = df.groupby('parent').amount.sum()
psumd = df.loc[df.parent.drop(df[df.parent == 0].index).unique()].amount.agg(lambda x: x - psum.loc[x.index])
psumb = df.loc[df.parent.drop(df[df.parent == 0].index).unique()].amount.agg(lambda x: x == psum.loc[x.index])

In [102]:
df_ = df.assign(
  is_equal=df.apply(lambda x: psumb.loc[x.parent if x.parent != 0 else x.name], axis=1),
  diff=df.apply(lambda x: psumd.loc[x.parent if x.parent != 0 else x.name], axis=1))

In [103]:
df_.to_csv('2022bkk-buget-item_with_parent.csv')

In [104]:
tf = df_[df_.is_equal == False]
bkkclean = pd.read_csv('bkk-bud_cleaned.csv', index_col='index')


In [111]:
len(tf)/len(df)

0.013475754253174406

In [105]:
bkkclean.groupby(['filename', 'page', 'line_num']).groups.__len__(),tf.groupby(['filename', 'page', 'line_num']).groups.__len__()

(43112, 381)

In [106]:

idx = []
bkkcleang = bkkclean.groupby(['filename', 'page', 'line_num'])
for x, g in tf.groupby(['filename', 'page', 'line_num']):
  idx += bkkcleang.get_group(x).index.tolist()

In [107]:
bkkclean.loc[idx].to_csv('bkk-toFix.csv')

In [108]:
# pd.read_csv('bkk-toFix.csv', index_col='index').to_dict('records')